In [54]:
import pandas as pd
import numpy as np
import re
import sys, getopt, os
import csv
import pickle
import copy

pd.set_option('display.max_rows', 500)

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline  
import seaborn as sns
sns.set_style("darkgrid")

%run SodaKick_download_functions.ipynb

import copy

In [55]:
#note xgboost can handle missing values internally
#list of parameters:
#https://xgboost.readthedocs.io/en/latest/parameter.html
#https://shengyg.github.io/repository/machine%20learning/2017/02/25/Complete-Guide-to-Parameter-Tuning-xgboost.html

#custom obj functions
#https://xgboost.readthedocs.io/en/latest/tutorials/custom_metric_obj.html
#https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters

In [56]:
import sklearn.datasets
import sklearn.metrics
from sklearn.model_selection import train_test_split

from sklearn.model_selection import RepeatedKFold, KFold
from sklearn.multioutput import MultiOutputRegressor

import xgboost as xgb
from xgboost import XGBRegressor

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.xgboost import TuneReportCheckpointCallback
from functools import partial 
from ray.tune.suggest.hyperopt import HyperOptSearch

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error as mse
#https://docs.ray.io/en/master/tune/tutorials/tune-xgboost.html

from hyperopt import hp

In [57]:
#https://towardsdatascience.com/quirky-keras-custom-and-asymmetric-loss-functions-for-keras-in-r-a8b5271171fe
def WSE(output, target, a=1.5, b=.5):
    loss = torch.mean(a*torch.minimum(torch.zeros(output.shape[1]),output - target)**2+\
                      b*torch.maximum(torch.zeros(output.shape[1]),output - target)**2)      
    return loss

def WSEl1(output, target, a=1.5, b=.5):
    loss = torch.mean(a*torch.abs(torch.minimum(torch.zeros(output.shape[1]),output - target))+\
                      b*torch.abs(torch.maximum(torch.zeros(output.shape[1]),output - target)))      
    return loss

def WSE2(output, target, a=1.5, b=.5):
    loss = np.mean(a*np.minimum(np.zeros(output.shape[0]),output - target)**2+\
                      b*np.maximum(np.zeros(output.shape[0]),output - target)**2)      
    return loss

def WSEl12(output, target, a=1.5, b=.5):
    loss = np.mean(a*np.abs(np.minimum(np.zeros(output.shape[0]),output - target))+\
                      b*np.abs(np.maximum(np.zeros(output.shape[0]),output - target)))      
    return loss

In [68]:
from sklearn import preprocessing

    
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_inp_a_210614.pkl', 'rb') as pk:
    inp=pickle.load(pk)
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_out_a_210614.pkl', 'rb') as pk:
    out=pickle.load(pk)     
    
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_inp_b_210614.pkl', 'rb') as pk:
    inpb=pickle.load(pk)
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_weight_out_b_210614.pkl', 'rb') as pk:
    outb=pickle.load(pk)     

inp=np.concatenate([inp,inpb])
out=np.concatenate([out,outb])

with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_lineup_a_210614.pkl', 'rb') as pk:
    lin=pickle.load(pk)
with open(r'/Users/federico comitani/GitHub/sodakick/data/10leagues_lineup_b_210614.pkl', 'rb') as pk:
    linb=pickle.load(pk)    

lin=np.concatenate([lin,linb])
lin2=[]
for i,l in enumerate(lin):
    lin2.append([x[0] for x in l])
lin=lin2

def normalize_mins(vec):
    for i in range(vec.shape[0]):
        vec[i][::8]=vec[i][::8]/90

def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

def NormalizeMatrix(data):   
    for i in range(data.shape[1]):
        data[:,i] = NormalizeData(data[:,i])
        
NormalizeMatrix(inp)
np.nan_to_num(inp, copy=False)

normalize_mins(out)

#scaler = preprocessing.StandardScaler().fit(inp)
#inp = scaler.transform(inp)

//miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in true_divide


In [13]:
from typing import Tuple

def WSE(predt: np.ndarray, dtrain: xgb.DMatrix) -> Tuple[str, float]:
    
    target = dtrain.get_label()
    predt[predt < -1] = -1 + 1e-6
    
    a=1.5
    b=.5
    
    elements = a*np.minimum(np.zeros(len(predt)),predt - target)**2+\
                      b*np.maximum(np.zeros(len(predt)),predt - target)**2
    
    return 'WSE', float(np.sqrt(np.sum(elements) / len(target)))

In [14]:
search_space = {
 "n_estimators": 100,
 "max_depth": hp.choice("max_depth",np.linspace(8,15,8, dtype=int)),
 #"max_depth": hp.choice("max_depth",np.linspace(1,15,8, dtype=int)),
 "min_child_weight": hp.choice("min_child_weight",[1, 2, 3, 4, 5]),
 "subsample": hp.choice("subsample",np.linspace(.6,.9,4)),
 #"subsample": hp.choice("subsample",np.linspace(.5,.9,5)),
 "eta": hp.choice("eta",[1e-2, 5e-2, 1e-1, 5e-1, 3e-1]),
 "colsample_bytree": hp.choice("colsample_bytree",np.linspace(0.1,.9,5)),
 "alpha": hp.randint("alpha", 3),
 #"alpha": hp.randint("alpha", 5),
 "lambda": hp.choice("lambda", np.linspace(0,10,6)),
 #"gamma": hp.choice("gamma",np.linspace(0,.9,4)),
 "objective": "reg:pseudohubererror",
 "eval_metric": "rmse", 
 "learning_rate": 1e-1, 
 }

x_train, x_test, y_train, y_test = train_test_split(
     inp, out, test_size=0.3, shuffle=True)

x_train, x_test, y_train, y_test = train_test_split(
     x_train, y_train, test_size=0.3, shuffle=True)

def train_xgb(config, col=0):
    
    evals=[]
    models=[]
    for k in range(48):
        models.append(XGBRegressor(**config))
        eval_set = [(x_train, y_train[:,col+8*k]),(x_test, y_test[:,col+8*k])]
        models[-1].fit(x_train, y_train[:,col+8*k], eval_metric="rmse", eval_set=eval_set, early_stopping_rounds = 3, verbose=False)    
        evals_result = models[-1].evals_result()   
        evals.append(evals_result['validation_1']['rmse'][-1]**2)
    
    #preds = model.predict(x_test,ntree_limit=model.get_booster().best_ntree_limit)    
    tune.report(rmse = np.sqrt(np.mean(evals)), done=True)    

def tune_xgb(search_space, col):
    
    search_alg = HyperOptSearch(space=search_space, metric="rmse", mode="min")

    # This will enable aggressive early stopping of bad trials.
    #scheduler = ASHAScheduler(
    #    max_t=20,  # 10 training iterations
    #    grace_period=5,
    #    reduction_factor=2)

    analysis = tune.run(
        partial(train_xgb, col=col),
        search_alg=search_alg,
        resources_per_trial={"cpu": 1},
        num_samples=25,
        #scheduler=scheduler,
        verbose=0,
        metric="rmse", mode="min")

    return analysis

In [9]:
#best_bst=[]
best_cfg=[]
for i in range(8):
    print('Feature #: '+str(i))
    analysis = tune_xgb(search_space, i)
    print(analysis)
    best_cfg.append(analysis.get_best_config())
    #best_cfg[-1]['file']=analysis.best_checkpoint
    #best_bst.append(get_best_model_checkpoint(analysis))
    print('\n')

Feature #: 0


2021-06-15 11:01:41,672	INFO services.py:1269 -- View the Ray dashboard at http://127.0.0.1:8265
2021-06-15 11:01:43,572	WARNING experiment.py:294 -- No name detected on trainable. Using DEFAULT.
2021-06-15 11:01:43,573	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.
2021-06-15 11:01:43,574	WARNING function_runner.py:545 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
2021-06-15 11:01:47,762	WARNING worker.py:1115 -- Warning: The actor ImplicitFunc has size 47666879 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
(pid=12899) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate ext



Feature #: 1


2021-06-15 12:36:38,590	WARNING worker.py:1115 -- Warning: The actor ImplicitFunc has size 47666879 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-06-15 12:36:38,654	WARNING util.py:162 -- The `start_trial` operation took 0.632 s, which may be a performance bottleneck.
2021-06-15 12:36:40,335	WARNING util.py:162 -- The `start_trial` operation took 0.502 s, which may be a performance bottleneck.
(pid=14433) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=14433)   "because it will generate extra copies and increase " +
2021-06-15 12:36:42,059	WARNING util.py:162 -- The `start_trial` operation took 0.616 s, which may be a performance bottleneck.
(pid=14437) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning:



Feature #: 2


...

2021-06-15 16:41:43,746	WARNING worker.py:1115 -- Warning: The actor ImplicitFunc has size 47666879 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
2021-06-15 16:41:43,804	WARNING util.py:162 -- The `start_trial` operation took 0.533 s, which may be a performance bottleneck.
(pid=17616) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=17616)   "because it will generate extra copies and increase " +
2021-06-15 16:41:47,129	WARNING util.py:162 -- The `start_trial` operation took 0.790 s, which may be a performance bottleneck.
(pid=17620) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption



Feature #: 6


2021-06-15 17:29:37,413	WARNING worker.py:1115 -- Warning: The actor ImplicitFunc has size 47666879 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
(pid=18170) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=18170)   "because it will generate extra copies and increase " +
(pid=18173) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=18173)   "because it will generate extra copies and increase " +
(pid=18176) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies an



Feature #: 7


2021-06-15 18:03:33,293	WARNING worker.py:1115 -- Warning: The actor ImplicitFunc has size 47666879 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.
(pid=18434) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=18434)   "because it will generate extra copies and increase " +
2021-06-15 18:03:36,176	WARNING util.py:162 -- The `start_trial` operation took 0.598 s, which may be a performance bottleneck.
(pid=18438) //miniconda3/lib/python3.7/site-packages/xgboost/data.py:106: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
(pid=18438)   "because it will generate extra copies and increase " +
2021-06-15 18:03:37,496	WARNING util.py:162 -- The `start

In [10]:
stats=pd.concat([pd.Series(bc) for bc in best_cfg], axis=1).T
print(stats

with open(r'/Users/federico comitani/GitHub/sodakick/data/best_cfg_weight_210614.pkl', 'wb') as file:
    pickle.dump(best_cfg,file)

,alpha,colsample_bytree,eta,eval_metric,lambda,learning_rate,max_depth,min_child_weight,n_estimators,objective,subsample
0,1,0.9,0.5,rmse,4.0,0.1,9,5,100,reg:pseudohubererror,0.9
1,2,0.5,0.05,rmse,10.0,0.1,9,1,100,reg:pseudohubererror,0.7
2,2,0.1,0.05,rmse,10.0,0.1,8,2,100,reg:pseudohubererror,0.8
3,0,0.5,0.01,rmse,8.0,0.1,8,5,100,reg:pseudohubererror,0.9
4,2,0.1,0.05,rmse,4.0,0.1,13,4,100,reg:pseudohubererror,0.6
5,0,0.1,0.3,rmse,10.0,0.1,14,4,100,reg:pseudohubererror,0.8
6,0,0.3,0.05,rmse,10.0,0.1,9,3,100,reg:pseudohubererror,0.8
7,1,0.7,0.01,rmse,8.0,0.1,8,3,100,reg:pseudohubererror,0.8


In [15]:
with open(r'/Users/federico comitani/GitHub/sodakick/data/best_cfg_weight_210614.pkl', 'rb') as file:
    best_cfg=pickle.load(file)

# one by one

In [17]:
x_train, x_test, y_train, y_test = train_test_split(inp, out, test_size=0.20, random_state=32)

models=[]
results=[]
epcs=[]
losses=[]
for num in range(48):
    print()
    print(num)
    print('---------')
    for num2 in range(8):
        print(num2, end='\t')
        
        tmpparms = best_cfg[num2]
        tmpparms.pop("file", None)    
        tmpparms['n_estimators']=1000
        #print(tmpparms)
        
        models.append(XGBRegressor(**tmpparms))
        models[-1].fit(x_train, y_train[:,num*8+num2],
            eval_set = [(x_train, y_train[:,num*8+num2]),(x_test, y_test[:,num*8+num2])],
            eval_metric = 'rmse',
            early_stopping_rounds = 5, verbose=False)
        
        results.append(models[-1].evals_result())
        best_iteration = models[-1].get_booster().best_ntree_limit
        
        epcs.append(models[-1].best_iteration)
        losses.append(models[-1].get_booster().best_score)
        
        print(epcs[-1],end='\t')
        print(losses[-1])
        
with open(r'/Users/federico comitani/GitHub/sodakick/data/models210614.pkl', 'wb') as file:
    pickle.dump(models,file)


0
---------
0	55	0.059536
1	101	0.004473
2	55	0.028741
3	41	0.183536
4	48	0.054274
5	52	0.038309
6	40	1.104921
7	87	1.640663

1
---------
0	85	0.136065
1	38	0.220587
2	27	0.238515
3	38	0.400771
4	42	0.100977
5	40	0.067609
6	45	0.045114
7	42	0.035665

2
---------
0	57	0.134602
1	34	0.316811
2	35	0.249988
3	30	0.38245
4	45	0.08121
5	47	0.054966
6	119	1e-06
7	120	0.00023

3
---------
0	183	0.138701
1	72	0.408627
2	36	0.35893
3	32	0.374625
4	39	0.072265
5	50	0.028702
6	119	1e-06
7	120	0.00023

4
---------
0	49	0.140247
1	126	0.454135
2	112	0.356994
3	32	0.371226
4	42	0.06631
5	43	0.046726
6	119	1e-06
7	120	0.00023

5
---------
0	64	0.144827
1	79	0.419522
2	64	0.318104
3	78	0.358871
4	44	0.072073
5	50	0.046826
6	119	1e-06
7	120	0.00023

6
---------
0	101	0.155019
1	34	0.360664
2	71	0.323307
3	30	0.35974
4	41	0.097266
5	47	0.039994
6	119	1e-06
7	120	0.00023

7
---------
0	104	0.133272
1	25	0.36062
2	72	0.314423
3	49	0.386611
4	44	0.08558
5	66	0.05302
6	119	1e-06
7	120	0.00023

8
---------
0

In [80]:
def revert_output(output,lineup=None,dropdummy=False):

    reframe=pd.DataFrame(copy.deepcopy(output.reshape(48,8)),
                 columns=['minutes','goals','assists','cards_yellow','cards_red','own_goals','goals_against','saves'])
    
    reframe[reframe<0] = 0
    reframe['team'] = 0
    reframe['team'].iloc[25:]=1
    reframe['minutes']*=90
    
    if lineup is not None:
        reframe.index=lineup
        if dropdummy:
            reframe.drop([x for x in reframe.index if x.startswith('dummy')], axis=0, inplace=True)
    
    #byteamframe=pd.concat([reframe.loc[[x for x in reframe.index if x in lineup[lineup['team']==0].index]].sum(axis=0),
    #                    reframe.loc[[x for x in reframe.index if x in lineup[lineup['team']==1].index]].sum(axis=0)], axis=1).T
    
    #byteamframe=pd.concat([reframe.iloc[:25,:].sum(axis=0),reframe.iloc[25:,:].sum(axis=0)], axis=1).T
    byteamframe=pd.concat([reframe[reframe['team']==0].sum(axis=0),reframe[reframe['team']==1].sum(axis=0)], axis=1).T
    
    return reframe, byteamframe[byteamframe.columns[1:-1]]

In [19]:
def multiprediction(inpt, models):
    
    preds=[]
    for model in models:
        preds.append(model.predict(inpt))
        
    return np.array(preds).T   


In [20]:
y_pred = multiprediction(x_test, models)

from sklearn.metrics import mean_squared_error as mse
mse(y_pred,y_test)

0.04787256994017458

In [21]:
print('Baseline WSE: {:.3f}'.format(WSE2(np.array([0]*y_train[0].shape[0]),y_train[0])))
print('Baseline WSE l1: {:.3f}'.format(WSEl12(np.array([0]*y_train[0].shape[0]),y_train[0])))
print('Baseline MSE: {:.3f}'.format(WSE2(np.array([0]*y_train[0].shape[0]),y_train[0], a=1, b=1)))
print('Baseline MSE l1: {:.3f}'.format(WSEl12(np.array([0]*y_train[0].shape[0]),y_train[0], a=1, b=1)))

print((y_train[1]-y_train[10]).sum())
print((y_train[50]-y_train[60]).sum())
print((y_train[100]-y_train[110]).sum())

print('Baseline WSE: {:.3f}'.format(WSE2(y_pred[0],y_train[0])))
print('Baseline WSE l1: {:.3f}'.format(WSEl12(y_pred[0],y_train[0])))
print('Baseline MSE: {:.3f}'.format(WSE2(y_pred[0],y_train[0], a=1, b=1)))
print('Baseline MSE l1: {:.3f}'.format(WSEl12(y_pred[0],y_train[0], a=1, b=1)))

print((y_pred[1]-y_pred[10]).sum())
print((y_pred[50]-y_pred[60]).sum())
print((y_pred[100]-y_pred[110]).sum())

Baseline WSE: 0.160
Baseline WSE l1: 0.152
Baseline MSE: 0.107
Baseline MSE l1: 0.102
5.0
-1.9222222222222225
-3.0
Baseline WSE: 0.096
Baseline WSE l1: 0.081
Baseline MSE: 0.124
Baseline MSE l1: 0.087
0.5528245
-1.4333359
10.385942


In [86]:
cats=['minutes','goals','assists','cards_yellow','cards_red','own_goals']+['goals_against','saves']

reframe, byteamframe = revert_output(y_pred[3],lin[0],dropdummy=False)
print(byteamframe)
reframe, byteamframe = revert_output(y_test[3],lin[0],dropdummy=False)
print(byteamframe)
reframe, byteamframe = revert_output(y_pred[3],lin[0],dropdummy=True)
print(byteamframe)
reframe, byteamframe = revert_output(y_test[3],lin[0],dropdummy=True)
print(byteamframe)

      goals   assists  cards_yellow  cards_red  own_goals  goals_against  \
0  0.820395  0.618932      1.601094   0.092380   0.029875       2.324677   
1  1.366893  0.913649      1.877508   0.138819   0.052846       0.022162   

      saves  
0  4.821275  
1  0.040595  
   goals  assists  cards_yellow  cards_red  own_goals  goals_against  saves
0    0.0      0.0           1.0        0.0        0.0            1.0    3.0
1    1.0      1.0           2.0        0.0        0.0            0.0    0.0
      goals   assists  cards_yellow  cards_red  own_goals  goals_against  \
0  0.819868  0.618473      1.599429   0.091769   0.029873       2.324676   
1  1.366367  0.913189      1.877507   0.138208   0.052845       0.022161   

      saves  
0  4.821044  
1  0.040365  
      goals   assists  cards_yellow  cards_red  own_goals  goals_against  \
0  0.819868  0.618473      1.599429   0.091769   0.029873       2.324676   
1  1.366367  0.913189      1.877507   0.138208   0.052845       0.022161   

 

//miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
//miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
//miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

In [87]:
reframe

,minutes,goals,assists,cards_yellow,cards_red,own_goals,goals_against,saves,team
Bartłomiej Drągowski,90.005051,0.000000,0.003322,0.021221,0.002269,0.001145,1.423849,2.442269,0
Cristiano Biraghi,86.387405,0.021630,0.038951,0.073678,0.007735,0.005199,0.002982,0.004894,0
Nikola Milenković,89.309402,0.014951,0.024842,0.168535,0.004567,0.002176,0.000001,0.000230,0
Giacomo Bonaventura,81.018738,0.108495,0.036273,0.125078,0.006843,0.001208,0.000001,0.000230,0
Franck Ribéry,81.949051,0.120268,0.105898,0.099802,0.005151,0.003839,0.000001,0.000230,0
Alfred Duncan,83.117157,0.042901,0.192894,0.086205,0.004955,0.000543,0.000001,0.000230,0
Gaetano Castrovilli,87.209503,0.128416,0.053619,0.177021,0.005422,0.003128,0.000001,0.000230,0
Christian Kouamé,79.297501,0.132839,0.013956,0.089459,0.005193,0.000000,0.000001,0.000230,0
Federico Ceccherini,86.015053,0.071172,0.031208,0.130973,0.003414,0.001207,0.000001,0.000230,0
Martín Cáceres,83.744110,0.017017,0.019891,0.266607,0.015384,0.000969,0.000001,0.000230,0


In [ ]:
# per player results
# or remove the missing when evaluating at the end